In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time, csv
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.relative_locator import locate_with

In [12]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\panag\AppData\Local\Temp/ipykernel_2272/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## 1. Football

In [13]:
football_url = 'https://en.stoiximan.gr/sport/soccer/'
driver.get(football_url)
cookies = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()

In [14]:
# Close Log in pop up window
x_button = driver.find_element(By.CSS_SELECTOR, '[class="sb-modal__close__btn uk-modal-close-default uk-icon uk-close"]').click()

In [25]:
# Click 24 hours button
button_24 = driver.find_element(By.CSS_SELECTOR, '[class="events-tabs-container__tab__item__button GTM-24"]').click()

In [38]:
### Scroll down the page and wait all values to load ###

# Get initial page height
last_height = driver.execute_script('window,scrollTo(0,document.body.scrollHeight)')

while True:
    # Wait for new values to load
    time.sleep(3)
    # Calculate the new height of the page and check if it has changed
    new_height = driver.execute_script('window,scrollTo(0,document.body.scrollHeight)')
    if new_height == last_height:
        # if the page height hasn't changed, exit the loop
        break
    else:
        # If the page height has changed, update the last height and continue scrolling
        last_height = new_height


# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
# By deafult is the football first
daily_coupon_body = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '[class="grid__column grid__column--fluid grid__column--main"]')))

# Extract footbal text
football_string = daily_coupon_body.text

In [53]:
# Create list from the initial string
initial_list = football_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Home Soccer Next 24 Hours Full Coupon', 'Soccer - Complete Coupon',
 'All', '3 hours', '12 hours','24 hours','By start time','By Competition',
 'Soccer - Matches in the next 24 hours','Matches','1','X','2','O/U 2.5','GG/NG','0%','1st leg']
list_1 = [x for x in initial_list if x not in remove_elements]

# Remove elements that start with "1st leg"
football_list = [x for x in list_1 if not x.startswith('1st leg:')]

In [75]:
# Create sublists based on date (matches)
match = [x for x in football_list if '/' in x]
index_match = [i for i,x in enumerate(football_list) if '/' in x]
sublists_matches = [football_list[i:j] for i, j in zip([0]+index_match, index_match + [len(football_list)])]

# Exclude the initial empty list from sublist_championships
sublists_matches = sublists_matches[1:]

# Remove the last element of each sublist (extra bets)
for sublist in sublists_matches:
    sublist.pop()

# Add extra elements for the missing bets
extra_element = 'No_bet'
for sublist in sublists_matches:
    if len(sublist) < 15:
        sublist.extend([extra_element] * (15 - len(sublist)))

In [81]:
columns = ['date', 'time', 'team1', 'team2', '1', 'X', '2', 'O', 'O_odds', 'U', 'U_odds', 'Yes', 'Yes_odds', 'No', 'No_odds']

df_football = pd.DataFrame(sublists_matches, columns=columns)

## 2. Basketball